The objective of this notebook is to create a dataset of cars in argentina. These cars are not any cars, but the ones rated by Doug Demuro in his (dataset)[https://www.dougdemuro.com/dougscore]. This is to show the reality of cars in argentina in relation to the rest of the world.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("data/dougScore/DougScore.xlsx")
pd.set_option('display.max_colwidth', None)

In [3]:
df.rename(columns={'Model ': 'Model'}, inplace=True)

In [4]:
# make sure all columns are strings
df["Model"] = df["Model"].astype(str)
df["Make"] = df["Make"].astype(str)

In [5]:
# For each of these cars, look them up on MercadoLibre API

import requests
import json

#xD, put it in a environment variable later
token = ""

# send the request
def getCarInfo(car):
    url = "https://api.mercadolibre.com/sites/MLA/search?q=" + car + "&access_token=" + token + "&category=MLA1743"
    response = requests.get(url)
    return response.json()


# test
resp = getCarInfo("toyota supra")["results"]
print(json.dumps(resp, indent=4, sort_keys=True))

[
    {
        "accepts_mercadopago": true,
        "address": {
            "city_id": "TUxBQ0NBUGZlZG1sYQ",
            "city_name": "Capital Federal",
            "state_id": "TUxBUENBUGw3M2E1",
            "state_name": "Capital Federal"
        },
        "attributes": [
            {
                "attribute_group_id": "",
                "attribute_group_name": "",
                "id": "ITEM_CONDITION",
                "name": "Condici\u00f3n del \u00edtem",
                "source": 1,
                "value_id": "2230581",
                "value_name": "Usado",
                "value_struct": null,
                "value_type": "list",
                "values": [
                    {
                        "id": "2230581",
                        "name": "Usado",
                        "source": 1,
                        "struct": null
                    }
                ]
            },
            {
                "attribute_group_id": "ADICIONALES",
             

In [6]:
arg_df = pd.DataFrame(columns=["Doug", "Make", "Model","Version", "Kilometers", "Engine", "Fuel Type", "Transmission", "Year", "City", "Price", "Currency", "Link"])


In [7]:
def sameBrand(a, original):

    if original in ["Mercedes-Benz", "Mercedes Benz", "Mercedes - Benz"]:
        return a in ["Mercedes-Benz", "Mercedes Benz", "Mercedes - Benz"]
    else:
        return a.lower() == original.lower()

def isSimilar(a, original):
    '''
    This function checks if the a string is similar to the original string
    '''
    a = a.lower()
    original = original.lower()
    
    if original in a:
        return True
    else:
        # priority check the first word
        a = a.split(" ")
        original = original.split(" ")
        if original[0] in a:
            # check if the rest are >= 0.5 in count
            count = 0
            for i in range(1, len(original)):
                if original[i] in a:
                    count += 1
            return count/len(original) >= 0.5
        else:
            return False

In [8]:
for row in range(len(df)):
    car = df.loc[row, ["Make", "Model", "Year"]]
    car_str = car["Make"] + " " + car["Model"] + " " + str(car["Year"])
    resp = getCarInfo(car_str)["results"]
    print("Original: ", car_str)
    for j in range(len(resp)):    
        valid_car = True
        for attr in resp[j]["attributes"]:
            # check if attributes coincide with the original car
            if attr["id"] == "BRAND":
                make = attr["value_name"]
            elif attr["id"] == "MODEL":
                model = attr["value_name"]
            elif attr["id"] == "TRIM":
                version = attr["value_name"]
            elif attr["id"] == "KILOMETERS":
                kilometers = attr["value_name"]
            elif attr["id"] == "ENGINE":
                engine_size = attr["value_name"]
            elif attr["id"] == "FUEL_TYPE":
                fuel_type = attr["value_name"]
            elif attr["id"] == "TRANSMISSION":
                transmission = attr["value_name"]
            elif attr["id"] == "VEHICLE_YEAR":
                year = int(attr["value_name"])
        
        

        # CHECKS
        if not sameBrand(make, car["Make"]):
            valid_car = False
        elif not isSimilar(model + " " + version, car["Model"]):
            valid_car = False


        # put this data in arg_df if the car is the same as the original
        if valid_car:
            city = resp[j]["location"]["city"]["name"]
            price = resp[j]["price"]
            currency_id = resp[0]["currency_id"]
            name = resp[j]["title"]
            link = resp[j]["permalink"]

            # append it to the dataframe
            arg_df.loc[len(arg_df)] = {"Doug": car_str, 
                                       "Make": make, "Model": model, "Version": version, 
                                       "Kilometers": kilometers, 
                                       "Engine": engine_size, "Fuel Type": fuel_type, 
                                       "Transmission": transmission, 
                                       "Year": year, 
                                       "City": city, 
                                       "Price": price, "Currency": currency_id, 
                                       "Link": link}


Original:  McLaren Speedtail 2020
Original:  McLaren F1 1994
Original:  Lucid Air Dream Edition 2022
Original:  GMC Hummer EV Pickup 2022
Original:  Rivian R1T 2021
Original:  Tesla Model S Plaid 2021
Original:  Bugatti Divo 2019
Original:  Bugatti Chiron Pur Sport 2021
Original:  Bugatti Chiron 2018
Original:  Ferrari Monza SP1 2020
Original:  Mercedes AMG E63 Wagon 2021
Original:  Rimac Nevera 2021
Original:  Koenigsegg Regera 2021
Original:  McLaren Sabre 2021
Original:  Ferrari SF90 Stradale 2021
Original:  Audi RS6 Avant 2021
Original:  Porsche Taycan Turbo S 2020
Original:  Ferrari LaFerrari 2015
Original:  Ferrari 488 Pista 2019
Original:  Koenigsegg Agera RS1 2018
Original:  Mercedes-Benz GT63S 4-Door 2019
Original:  Mercedes-Benz E63S AMG Wagon 2018
Original:  Porsche 918 Spyder 2015
Original:  Tesla Model X Plaid 2022
Original:  Porsche Taycan GTS Sport Turismo 2022
Original:  Porsche Taycan Turbo CrossTurismo 2021
Original:  Audi RS E-Tron GT 2022
Original:  Ferrari 812 GTS 

In [9]:
arg_df.head()

,Doug,Make,Model,Version,Kilometers,Engine,Fuel Type,Transmission,Year,City,Price,Currency,Link
0,Lamborghini Huracan Evo Spyder 2020,Lamborghini,Huracan Evo,Spyder Malek Fara,1001 km,V10 5.2,Nafta,Automática secuencial,2023,Capital Federal,123321,USD,https://auto.mercadolibre.com.ar/MLA-1530995974-lamborghini-huracan-evo-spyder-no-ferrari-porsche-malek-fara-_JM
1,Lamborghini Urus 2019,Lamborghini,Urus,Malek Fara,1001 km,V8 Biturbo 650cv,Nafta,Automática secuencial,2023,Capital Federal,123321,USD,https://auto.mercadolibre.com.ar/MLA-1393359903-lamborghini-urus-no-ferrari-no-porsche-malek-fara-_JM
2,Lamborghini Huracan 2015,Lamborghini,Huracan Evo,Spyder Malek Fara,1001 km,V10 5.2,Nafta,Automática secuencial,2023,Capital Federal,123321,USD,https://auto.mercadolibre.com.ar/MLA-1530995974-lamborghini-huracan-evo-spyder-no-ferrari-porsche-malek-fara-_JM
3,Ferrari F12 Berlinetta 2015,Ferrari,F12,F12 Berlinetta,3300 km,3.0 V6 Turbo 340CV,Nafta,Automática,2018,Capital Federal,900000,ARS,https://auto.mercadolibre.com.ar/MLA-1527051654-ferrari-f12-f12-berlinetta-_JM
4,Ford GT 2019,Ford,Mustang,5.0 Gt 421cv,10300 km,5.0,Nafta,Manual,2019,San Justo,78000,USD,https://auto.mercadolibre.com.ar/MLA-1543098416-ford-mustang-2017-50-gt-421cv-_JM


In [10]:
arg_df["Price"] = arg_df["Price"].astype(int)
arg_df["Year"] = arg_df["Year"].astype(int)

In [11]:
# Number of unique dougscore cars
print(len(df))
print(arg_df["Doug"].nunique())
print("Percentage of cars found: ", round(arg_df["Doug"].nunique()/len(df)*100), "%")

584
92
Percentage of cars found:  16 %


In [12]:
# look for repeated entries (don't take the link into account)
arg_df[arg_df.duplicated(subset=arg_df.columns[-1], keep=False)].head()

,Doug,Make,Model,Version,Kilometers,Engine,Fuel Type,Transmission,Year,City,Price,Currency,Link
0,Lamborghini Huracan Evo Spyder 2020,Lamborghini,Huracan Evo,Spyder Malek Fara,1001 km,V10 5.2,Nafta,Automática secuencial,2023,Capital Federal,123321,USD,https://auto.mercadolibre.com.ar/MLA-1530995974-lamborghini-huracan-evo-spyder-no-ferrari-porsche-malek-fara-_JM
2,Lamborghini Huracan 2015,Lamborghini,Huracan Evo,Spyder Malek Fara,1001 km,V10 5.2,Nafta,Automática secuencial,2023,Capital Federal,123321,USD,https://auto.mercadolibre.com.ar/MLA-1530995974-lamborghini-huracan-evo-spyder-no-ferrari-porsche-malek-fara-_JM
4,Ford GT 2019,Ford,Mustang,5.0 Gt 421cv,10300 km,5.0,Nafta,Manual,2019,San Justo,78000,USD,https://auto.mercadolibre.com.ar/MLA-1543098416-ford-mustang-2017-50-gt-421cv-_JM
5,Ford GT 2019,Ford,Mustang,5.0 Gt 421cv,22000 km,5.0,Nafta,Manual,2017,Pilar,80000,USD,https://auto.mercadolibre.com.ar/MLA-1530392588-ford-mustang-50-gt-421cv-2019-_JM
6,Porsche 911 Turbo S (992) 2021,Porsche,911,3.8 Turbo S,77000 km,3.8,Nafta,Manual,1998,San Isidro,299000,USD,https://auto.mercadolibre.com.ar/MLA-1533910398-porsche-911-turbo-930-1980-_JM


This is somewhat common in MercadoLibre, specially in cars. Some cars have duplicate publications and they have the same specs. These will be deleted from the dataset.

In [13]:
# delete repeated entries
arg_df = arg_df.drop_duplicates(subset=arg_df.columns[:-1], keep="first")

In [14]:
# Merge the two datasets. For each entry in the arg_df, add all the columns that correspond to the Doug column

# this will need to be an artificial merge, since the Doug column (in arg_df) is the combination of Make + Model in the df
df["Year"] = df["Year"].astype(str)
# create a "Doug" column on the df that is the combination of Make + Model + Year
df["Doug"] = df["Make"] + " " + df["Model"] + " " + df["Year"]
# change \n to " "
df["Doug"] = df["Doug"].str.replace("\n", " ")

merged_df = pd.merge(arg_df, df, on="Doug", how="left")

In [16]:
merged_df.to_csv("ArgentinaCars.csv", index=False)